## 基本使用

对于 tf.constant 创建的变量，要计算，需要 watch；如果不 watch，计算出来的是 None

In [87]:
import tensorflow as tf
x = tf.constant(3.0)

with tf.GradientTape() as tape:
    y = x * x
dx = tape.gradient(y, x)
print(dx)

None


In [88]:
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x * x
dx = tape.gradient(y, x)
print(dx)

tf.Tensor(6.0, shape=(), dtype=float32)


对于 tf.Variable 创建出来的，会自动加入求导列表中，无需 watch

In [89]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x * x
dx = tape.gradient(y, x)
print(dx)

tf.Tensor(6.0, shape=(), dtype=float32)


## Persistent 参数

默认情况下，tape 只能用来计算一次导数。第二次计算导数会报错

In [90]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
    y = x * x
dx = tape.gradient(y, x)
new_dx = tape.gradient(y, x)
print(dx)
print(new_dx) # GradientTape.gradient can only be called once on non-persistent tapes.

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

使用 persistent=True 可以让 tape 保持导数，可以多次计算。

注意： 这时需要手动 GC。


In [91]:
x = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as tape:
    y = x * x
dx = tape.gradient(y, x)
new_dx = tape.gradient(y, x)
print(dx) 
print(new_dx) # GradientTape.gradient can only be called once on non-persistent tapes.
del tape

tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


## 计算高阶导数

In [92]:
x = tf.Variable(3.0)

with tf.GradientTape() as t1:
    with tf.GradientTape() as t2:
        y = x * x
    dx = t2.gradient(y, x)
ddx = t1.gradient(dx, x)

print(dx) 
print(ddx) # GradientTape.gradient can only be called once on non-persistent tapes.

tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
